In [147]:
import pandas as pd

In [148]:
def split_years(df):
    df['year'] = df['resources.description'].str.extract(r'(\d{4})')
    df['resources.description'] = df['resources.description'].str.replace(r'\d{4}', '', regex=True)
    df['year'] = df['year'].fillna(0).astype(int)

    df['year_2'] = df['resources.description'].str.extract(r'(\d{2})')
    df['resources.description'] = df['resources.description'].str.replace(r'\d{2}', '', regex=True)
    df['year_2'] = pd.to_datetime(df['year_2'], format='%y').dt.strftime('%Y')
    df['year_2'] = df['year_2'].fillna(0).astype(int)

    df['year'] = df['year'] + df['year_2']

    df = df.drop(['year_2'], axis=1)

    return df


In [149]:
def clean_Subscriptions_data(subset):
    subset['resources.description'] = subset['resources.description'].replace({'\\(|\\)': '', '  ': ' '}, regex=True)
    subset['resources.description'] = subset['resources.description'].str.split(':', expand=False)
    subset['schedule'] = schedule
    subset['section_month'] = subset['resources.description'].str[1]
    subset['section_month'] = subset['section_month'].str.split(' ', expand=False)
    subset['section_month'] = subset['section_month'].apply(lambda x: [item for item in x if item])
    subset['section'] = subset['section_month'].str[0]
    subset['month'] = subset['section_month'].str[1]
    subset = subset.drop(['section_month'], axis=1)

    return subset


In [150]:
def clean_data(file_path):
    df = pd.read_csv(file_path)

    columns_to_keep = ['resources.description', 'gross_amount', 'gocardless_fees', 'app_fees', 'net_amount', 'payouts.arrival_date', 'payments.metadata.Member', 'payments.metadata.References']
    df = df[columns_to_keep]
    df = split_years(df)

    payment_schedules = ['Subscriptions', 'Activities', ]

    dfs = []

    for schedule in payment_schedules:
        subset = df[df['resources.description'].str.contains(schedule)].copy()
        if schedule == 'Subscriptions':
            subset = clean_Subscriptions_data(subset)
        

        dfs.append(subset)

    df = pd.concat(dfs, ignore_index=True)







    return df

df = clean_data(file_path='inputs_3/payout_transactions_reconciliation-export-EX00036AZABPPE.csv')
df.sample(5)


,resources.description,gross_amount,gocardless_fees,app_fees,net_amount,payouts.arrival_date,payments.metadata.Member,payments.metadata.References,year,schedule,section,month
91,"[Monthly Subscriptions, Cubs February ]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Isabelle Blakeway (2535179),684665 - 59655 - 15536,2024,Subscriptions,Cubs,February
130,"Activities (Camps, Sleepovers etc): Scouts ( L...",58.0,0.71,1.13,56.16,2024-02-12 00:00:00,Ethan Lovely (1696666),730026 - 57593 - 19515,2024,NaN,NaN,NaN
58,"[Monthly Subscriptions, Squirrels January]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Evie Gullick (2737608),716136 - 67369 - 69829,2024,Subscriptions,Squirrels,January
34,"[Monthly Subscriptions, Beavers November]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Freya Hayes (2428327),697482 - 65921 - 16992,2023,Subscriptions,Beavers,November
99,"[Monthly Subscriptions, Scouts February]",12.0,0.29,0.23,11.48,2024-02-12 00:00:00,Oliver Goddard (1214880),714274 - 57594 - 19515,2024,Subscriptions,Scouts,February
